# Learning Goals

## Supervised Finetuning (SFT)

Often we want to adapt or customize foundation models to be more performant on our specific task. Fine-tuning refers to how we can modify the weights of a pre-trained foundation model with additional custom data. Supervised fine-tuning (SFT) refers to unfreezing all the weights and layers in our model and training on a newly labeled set of examples. We can fine-tune to incorporate new, domain-specific knowledge, or teach the foundation model what type of response to provide. One specific type of SFT is also referred to as “instruction tuning” where we use SFT to teach a model to follow instructions better. In this playbook will demostrate how to perform SFT with Llama3-8b using NeMo 2.0.

## NeMo2.0

In NeMo 1.0, the main interface for configuring experiments is through YAML files. This approach allows for a declarative way to set up experiments, but it has limitations in terms of flexibility and programmatic control. NeMo 2.0 is an update on the NeMo Framework which introduces several significant improvements over its predecessor, NeMo 1.0, enhancing flexibility, performance, and scalability.

- Python-Based Configuration - NeMo 2.0 transitions from YAML files to a Python-based configuration, providing more flexibility and control. This shift makes it easier to extend and customize configurations programmatically.

- Modular Abstractions - By adopting PyTorch Lightning’s modular abstractions, NeMo 2.0 simplifies adaptation and experimentation. This modular approach allows developers to more easily modify and experiment with different components of their models.

- Scalability - NeMo 2.0 seamlessly scaling large-scale experiments across thousands of GPUs using NeMo-Run, a powerful tool designed to streamline the configuration, execution, and management of machine learning experiments across computing environments.

By adopting PyTorch Lightning’s modular abstractions, NeMo 2.0 makes it easy for users to adapt the framework to their specific use cases and experiment with various configurations. This section offers an overview of the new features in NeMo 2.0 and includes a migration guide with step-by-step instructions for transitioning your models from NeMo 1.0 to NeMo 2.0.


## Software Requirements

1. Use the latest [NeMo Framework Training container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo/tags) . Note that you must be logged in to the container registry to view this page.

2. This notebook uses the container: `nvcr.io/nvidia/nemo:dev`  #TODO: FIX CONTAINER  


## Hardware Requirements

- Minimum 8xA100 80G (1 node) for SFT on 7B and 13B

- SFT can be run on all (7B/13B/70B) model sizes on multiple nodes


## Data
Databricks-dolly-15k is an open-source dataset created by the collaborative efforts of Databricks employees. It consists of high-quality human-generated prompt/response pairs specifically designed for instruction tuning LLMs. These pairs cover a diverse range of behaviors, from brainstorming and content generation to information extraction and summarization. 

For more information, refer to [databricks-dolly-15k | Hugging Face](https://huggingface.co/datasets/databricks/databricks-dolly-15k)

# Step 0: Go inside docker container

Here is a demo of starting and go inside the container on DGX Cloud. '

Otherwise, you can start and enter the dev container by:  #TODO: FIX CONTAINER
```
docker run --gpus device=1 --shm-size=2g --net=host --ulimit memlock=-1 --rm -it -v ${PWD}:/workspace -w /workspace -v ${PWD}/results:/results nvcr.io/nvidia/nemo:dev bash

```


# Step 1: Import HuggingFace checkpoint
First request download permission from Meta and Hugging Face. Login through `huggingface-cli` using your Huggingface token before importing llama3 models. 

```
$ huggingface-cli login
```

Once you are logged in, NeMo 2.0 will automatically import the Hugging Face model and start training. There is no need to manully convert to NeMo checkpoint format.

Let's first import needed python modules:

In [ ]:
from nemo import lightning as nl
from nemo.collections import llm
from megatron.core.optimizer import OptimizerConfig
import torch
import pytorch_lightning as pl

## Step 2: Prepare data and customize DataModule

We will be using Databricks-dolly-15k for this notebook. NeMo 2.0 already provides a `DollyDataModule`. Example usage:

In [ ]:

def dolly() -> pl.LightningDataModule:
    return llm.DollyDataModule(seq_length=2048, micro_batch_size=2, global_batch_size=8, num_workers=0)

To use your own data, you will need to create a custom `DataModule`. This involvse extending the base class `FineTuningDataModule`, so that you have access to existing data handling logic such as packed sequence. Here we walk you through the process step by step using the already existing [`DollyDataModule`](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/llm/gpt/data/dolly.py) as an example. 

### 1. Subclass the FineTuningDataModule
You need to extend `FineTuningDataModule` if you're fine-tuning NeMo models. This provides access to existing data handling logic, such as packed sequences and tokenization. The `data_root` parameter is where you store your generated `train/validation/test.jsonl` in NeMo format. How `DollyDataModule` does it:

In [ ]:
from datasets import load_dataset
from typing import TYPE_CHECKING, List, Optional
from nemo.collections.common.tokenizers import TokenizerSpec
from nemo.lightning.io.mixin import IOMixin
from nemo.collections.llm.gpt.data.fine_tuning import FineTuningDataModule
import json
from nemo.utils import logging
import numpy as np
import shutil

class DollyDataModule(FineTuningDataModule, IOMixin):
    def __init__(
        self,
        seq_length: int = 2048,
        tokenizer: Optional["TokenizerSpec"] = None,
        micro_batch_size: int = 4,
        global_batch_size: int = 8,
        rampup_batch_size: Optional[List[int]] = None,
        force_redownload: bool = False,
        delete_raw: bool = True,
        seed: int = 1234,
        memmap_workers: int = 1,
        num_workers: int = 8,
        pin_memory: bool = True,
        persistent_workers: bool = False,
        pad_to_max_length: bool = False,
        packed_sequence_size: int = -1,
    ):
        self.force_redownload = force_redownload
        self.delete_raw = delete_raw

        super().__init__(
            dataset_root=get_dataset_root("dolly"),
            seq_length=seq_length,
            tokenizer=tokenizer,
            micro_batch_size=micro_batch_size,
            global_batch_size=global_batch_size,
            rampup_batch_size=rampup_batch_size,
            seed=seed,
            memmap_workers=memmap_workers,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=persistent_workers,
            pad_to_max_length=pad_to_max_length,
            packed_sequence_size=packed_sequence_size,
        )

### 2. Override the `prepare_data` Method

The prepare_data method is responsible for downloading and preprocessing data if needed. If the dataset is already downloaded, you can skip this step.



In [ ]:
def prepare_data(self) -> None:
    # if train file is specified, no need to do anything
    if not self.train_path.exists() or self.force_redownload:
        dset = self._download_data()
        self._preprocess_and_split_data(dset)
    super().prepare_data()

### 3. Implement Data Download and Preprocessing Logic

If your dataset requires downloading or preprocessing, implement this logic within helper methods. Skip the download part if it's not needed.

In [ ]:
def _download_data(self):
    logging.info(f"Downloading {self.__class__.__name__}...")
    return load_dataset(
        "databricks/databricks-dolly-15k",
        cache_dir=str(self.dataset_root),
        download_mode="force_redownload" if self.force_redownload else None,
    )

def _preprocess_and_split_data(self, dset, train_ratio: float = 0.80, val_ratio: float = 0.15):
    logging.info(f"Preprocessing {self.__class__.__name__} to jsonl format and splitting...")

    test_ratio = 1 - train_ratio - val_ratio
    save_splits = {}
    dataset = dset.get('train')
    split_dataset = dataset.train_test_split(test_size=val_ratio + test_ratio, seed=self.seed)
    split_dataset2 = split_dataset['test'].train_test_split(
        test_size=test_ratio / (val_ratio + test_ratio), seed=self.seed
    )
    save_splits['training'] = split_dataset['train']
    save_splits['validation'] = split_dataset2['train']
    save_splits['test'] = split_dataset2['test']

    for split_name, dataset in save_splits.items():
        output_file = self.dataset_root / f"{split_name}.jsonl"
        with output_file.open("w", encoding="utf-8") as f:
            for example in dataset:
                context = example["context"].strip()
                if context != "":
                    # Randomize context and instruction order.
                    context_first = np.random.randint(0, 2) == 0
                    if context_first:
                        instruction = example["instruction"].strip()
                        assert instruction != ""
                        _input = f"{context}\n\n{instruction}"
                        _output = example["response"]
                    else:
                        instruction = example["instruction"].strip()
                        assert instruction != ""
                        _input = f"{instruction}\n\n{context}"
                        _output = example["response"]
                else:
                    _input = example["instruction"]
                    _output = example["response"]

                f.write(json.dumps({"input": _input, "output": _output, "category": example["category"]}) + "\n")

        logging.info(f"{split_name} split saved to {output_file}")

    if self.delete_raw:
        for p in self.dataset_root.iterdir():
            if p.is_dir():
                shutil.rmtree(p)
            elif '.jsonl' not in str(p.name):
                p.unlink()

The original example in Dolly dataset looks like:
```
{'instruction': 'Extract all the movies from this passage and the year they were released out. Write each movie as a separate sentence', 'context': "The genre has existed since the early years of silent cinema, when Georges Melies' A Trip to the Moon (1902) employed trick photography effects. The next major example (first in feature length in the genre) was the film Metropolis (1927). From the 1930s to the 1950s, the genre consisted mainly of low-budget B movies. After Stanley Kubrick's landmark 2001: A Space Odyssey (1968), the science fiction film genre was taken more seriously. In the late 1970s, big-budget science fiction films filled with special effects became popular with audiences after the success of Star Wars (1977) and paved the way for the blockbuster hits of subsequent decades.", 'response': 'A Trip to the Moon was released in 1902. Metropolis came out in 1927. 2001: A Space Odyssey was released in 1968. Star Wars came out in 1977.', 'category': 'information_extraction'}
```
After the preprocessing logic, the data examples are transformed into NeMo format, as below:
```
{'input': "Extract all the movies from this passage and the year they were released out. Write each movie as a separate sentence\n\nThe genre has existed since the early years of silent cinema, when Georges Melies' A Trip to the Moon (1902) employed trick photography effects. The next major example (first in feature length in the genre) was the film Metropolis (1927). From the 1930s to the 1950s, the genre consisted mainly of low-budget B movies. After Stanley Kubrick's landmark 2001: A Space Odyssey (1968), the science fiction film genre was taken more seriously. In the late 1970s, big-budget science fiction films filled with special effects became popular with audiences after the success of Star Wars (1977) and paved the way for the blockbuster hits of subsequent decades.", 'output': 'A Trip to the Moon was released in 1902. Metropolis came out in 1927. 2001: A Space Odyssey was released in 1968. Star Wars came out in 1977.', 'category': 'information_extraction'}
```
Each data example is saved as a json string as one line in the `train/validation/test.jsonl` file, under `data_root` directory you specified earlier.

## Step 3: Run SFT with NeMo 2.0 API 

The following python script utilizes NeMo 2.0 API to perform SFT. In this script we are configuring the following components for training. These components are similar between SFT and PEFT. SFT and PEFT both uses `llm.finetune` API. To switch from PEFT to SFT you just need to remove `peft` parameter.

### Trainer
NeMo 2.0 Trainer works simiarly to Pytorch Lightning trainer. You can specify to use MegatronStrategy as your model parallel strategy to use NVIDIA's Megatron-LM framework and pass in configurations as below:



In [ ]:

def trainer(devices=1) -> nl.Trainer:
    strategy = nl.MegatronStrategy(
        tensor_model_parallel_size=1,
    )

    return nl.Trainer(
        devices=devices,
        max_steps=40,
        accelerator="gpu",
        strategy=strategy,
        plugins=nl.MegatronMixedPrecision(precision="bf16-mixed"),
        log_every_n_steps=1,
        limit_val_batches=2,
        val_check_interval=2,
        num_sanity_val_steps=0,
    )



### Logger
Configure your training steps, output directories and logging through `NeMoLogger`. In the following example, the experiment output will be saved at `./results/nemo2_sft`.



In [ ]:
def logger() -> nl.NeMoLogger:
    ckpt = nl.ModelCheckpoint(
        save_last=True,
        every_n_train_steps=10,
        monitor="reduced_train_loss",
        save_top_k=1,
        save_on_train_epoch_end=True,
        save_optim_on_train_end=True,
    )

    return nl.NeMoLogger(
        name="nemo2_sft",
        log_dir="./results",
        use_datetime_version=False,
        ckpt=ckpt,
        wandb=None
    )



### Optimizer
In the following example, we will be using distributed adam optimizer, and pass in optimizer configuration through `OptimizerConfig`: 




In [ ]:
def adam_with_cosine_annealing() -> nl.OptimizerModule:
    return nl.MegatronOptimizerModule(
        config=OptimizerConfig(
            optimizer="adam",
            lr=0.0001,
            adam_beta2=0.98,
            use_distributed_optimizer=True,
            clip_grad=1.0,
            bf16=True,
        ),
    )

### Base Model
We will perform SFT on top of Llama3-8b so we create a `LlamaModel` to pass to finetune API.

In [ ]:
def llama3_8b() -> pl.LightningModule:
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
    return llm.LlamaModel(llm.Llama3Config8B(), tokenizer=tokenizer)

### AutoResume
In NeMo 2.0 we can directly pass in Llama3-8b's Hugging Face ID to start SFT without manually converting it into NeMo checkpoint format like in NeMo 1.0.

In [ ]:

def resume() -> nl.AutoResume:
    return nl.AutoResume(
        restore_config=nl.RestoreConfig(
            path="hf://meta-llama/Meta-Llama-3-8B"
        ),
        resume_if_exists=True,
    )


### NeMo 2.0 finetun API
Using all the components we created above, we can call NeMo 2.0 finetun API:
```
llm.finetune(
    model=llama3_8b(),
    data=dolly(),
    trainer=trainer(),
    log=logger(),
    optim=adam_with_cosine_annealing(),
    resume=resume(),
)
```
Below is a python script that you can save as a file e.g. `nemo2-sft.py`, and run SFT training, using all components we created above and NeMo 2.0 finetune API. The script cannot be directly executed in interactive environment like a notebook. We can execute by `torchrun --nproc_per_node=<NUM_GPU> nemo2-sft.py` when multiple GPU is used.

In [ ]:
from nemo import lightning as nl
from nemo.collections import llm
from megatron.core.optimizer import OptimizerConfig
import torch
import pytorch_lightning as pl


def trainer(devices=1) -> nl.Trainer:
    strategy = nl.MegatronStrategy(
        tensor_model_parallel_size=1,
    )

    return nl.Trainer(
        devices=devices,
        max_steps=40,
        accelerator="gpu",
        strategy=strategy,
        plugins=nl.MegatronMixedPrecision(precision="bf16-mixed"),
        log_every_n_steps=1,
        limit_val_batches=2,
        val_check_interval=2,
        num_sanity_val_steps=0,
    )


def logger() -> nl.NeMoLogger:
    ckpt = nl.ModelCheckpoint(
        save_last=True,
        every_n_train_steps=10,
        monitor="reduced_train_loss",
        save_top_k=1,
        save_on_train_epoch_end=True,
        save_optim_on_train_end=True,
    )

    return nl.NeMoLogger(
        name="nemo2_sft",
        log_dir="./results",
        use_datetime_version=False,
        ckpt=ckpt,
        wandb=None
    )


def adam_with_cosine_annealing() -> nl.OptimizerModule:
    return nl.MegatronOptimizerModule(
        config=OptimizerConfig(
            optimizer="adam",
            lr=0.0001,
            adam_beta2=0.98,
            use_distributed_optimizer=True,
            clip_grad=1.0,
            bf16=True,
        ),
    )

def dolly() -> pl.LightningDataModule:
    return llm.DollyDataModule(seq_length=2048, micro_batch_size=2, global_batch_size=8, num_workers=0)



def llama3_8b() -> pl.LightningModule:
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
    return llm.LlamaModel(llm.Llama3Config8B(), tokenizer=tokenizer)

def resume() -> nl.AutoResume:
    return nl.AutoResume(
        restore_config=nl.RestoreConfig(
            path="hf://meta-llama/Meta-Llama-3-8B"
        ),
        resume_if_exists=True,
    )

if __name__ == '__main__':
    llm.finetune(
        model=llama3_8b(),
        data=dolly(),
        trainer=trainer(),
        log=logger(),
        optim=adam_with_cosine_annealing(),
        resume=resume(),
    )

## Step 4 Evaluation ##TODO: depending on NeMo 2.0 llm generation API

## Optional: Launch with [NeMoRun](https://github.com/NVIDIA/NeMo-Run)
Alternatively, we could use launch SFT jobs using existing [recipes](https://github.com/NVIDIA/NeMo/tree/main/nemo/collections/llm/recipes) from NeMo-Run. A recipe in NeMo is a python file that defines a complete configuration for training or fine-tuning an LLM. Each recipe typically includes:
1. Model configuration: Defines the architecture and hyperparameters of the LLM.
2. Training configuration: Specifies settings for the PyTorch Lightning Trainer, including distributed training strategies.
3. Data configuration: Sets up the data pipeline, including batch sizes and sequence lengths.
4. Optimization configuration: Defines the optimizer and learning rate schedule.
5. Logging and checkpointing configuration: Specifies how to save model checkpoints and log training progress.

Recipes are designed to be modular and extensible, allowing users to easily customize settings for their specific use cases.


NeMo-Run is a powerful tool designed to streamline the configuration, execution, and management of machine learning experiments across various computing environments. NeMo-Run is responsible for experiment configuration, execution and management. Here is an example for launch a recipe using NeMo-Run using local executor. ##TODO: the default finetuning recipe include LoRA

In [ ]:
## TODO: Pretrain with tp1pp1cp2 doesn't work. Pretrain with tp4pp1cp2 works. Finetuning recipe doesn't work
import nemo_run as run
from nemo.collections import llm

recipe = llm.llama3_8b.finetune_recipe(name="llama3-8b-pretrain", dir="exp/nemorun_ft", num_nodes=1, num_gpus_per_node=2)
env_vars = {
    "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
    "NCCL_NVLS_ENABLE": "0",
    "NVTE_DP_AMAX_REDUCE_INTERVAL": "0",
    "NVTE_ASYNC_AMAX_REDUCTION": "1",
    "NVTE_FUSED_ATTN": "0",
}
local_executor = run.LocalExecutor(ntasks_per_node=8, launcher="torchrun", env_vars=env_vars)
run.run(recipe, executor=local_executor)